In [1]:
import torch
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertPreTrainedModel, BertForPreTraining, BertForMaskedLM
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook
import os
import re
RUBERT_PATH = 'C:\\Users\Bekzat\\Documents\\GitHub\\NLP_winter_2020\\Lab_#3\\ru_conversational_cased_L-12_H-768_A-12_pt'
modelpath = os.path.join(RUBERT_PATH,'pytorch_model.bin')

In [2]:
os.path.isfile(os.path.join(RUBERT_PATH,'pytorch_model.bin'))

True

In [3]:
tokenizer = BertTokenizer.from_pretrained(RUBERT_PATH, do_lower_case=False)
config = BertConfig.from_json_file(os.path.join(RUBERT_PATH,'bert_config.json'))
model = BertForPreTraining.from_pretrained(modelpath, config=config)
model.eval()
from torch import load
di = load(modelpath)

In [4]:
# due to limits of computer power, the number of data set is reduced to 900

In [5]:
def mean_calc(text): 
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    segments_ids[0] = 0
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    predictions = model(tokens_tensor, token_type_ids=segments_tensors)
    array_of_thensors = []
    for i in range(len(tokenized_text)):
        array_of_thensors.append((predictions[0][0][i].detach().numpy()))
       
    mean = np.mean(array_of_thensors, axis = 0)
    return mean

In [6]:
df= pd.DataFrame()
df = pd.read_csv('texts_train.txt', sep="\t", encoding='UTF-8',header=0)
df['score'] = pd.read_csv('scores_train.txt', sep="\t", encoding='UTF-8',header=0)
df.columns = ['text', 'score']
print(df.head())

                                                text  score
0  думал, что будет лучше идея очень интересна - ...      7
1  с творчеством Головачева я познакомился посред...     10
2  то-то я и в большое неудовольствие прочитал "А...      5
3  как мне показалось местами сильно смахивает на...      6
4  от первой части книги просто оторваться не мог...      9


In [7]:
unique_grades = np.sort(df['score'].unique())
print('original range of scores: ',unique_grades)
unique_grades  = [1,5,10]
print('new range of scores: ',unique_grades)

original range of scores:  [ 1  2  3  4  5  6  7  8  9 10]
new range of scores:  [1, 5, 10]


In [8]:
df_new = pd.DataFrame()
comment = []
grade = []

for grades in unique_grades:
    counter = 0
    for i in range(len(df)):
    #for i in range(000/(len(unique_grades))):
        if (counter<900):
            if (grades == df['score'][i]):
                comment.append(df['text'][i])
                grade.append(df['score'][i])
                counter = counter + 1 
df_new['text'] = comment
# lower the range of classifiers to ranging 1-3, where 1 is negative and 3 positive
df_new['score'] = grade
df_new['score'] = df_new['score'].replace(5, 2)
df_new['score'] = df_new['score'].replace(10, 3)
print('substituted range of scores: ',np.sort(df_new['score'].unique()))
print(df_new.head())

substituted range of scores:  [1 2 3]
                                                text  score
0  Хотелось долго плеваться после того как я с тр...      1
1  ИМХО, в серии о Свароге Бушков исписался. Если...      1
2  Много был наслышан об этом авторе и вот взялся...      1
3  Да, тяжело читать безграмотную фэнтези. Хотя, ...      1
4  Попытка свести сюжет предыдущих книг в единое ...      1


In [9]:
df_new.to_csv('treeclasses_sorted_comments.csv', sep = '|') #save this dataset to csv

In [10]:
array_of_mean_tensors = []
grade = []
comment = []
errors = 0
for i in range(len(df_new)):
    try: 
        array_of_mean_tensors.append(mean_calc(df_new['text'][i]))
        grade.append(df_new['score'][i])
        comment.append(df_new['text'][i])
    except:
        #in some cases, from my point of view, when comment is too large, the data has not processed, and it cause some errors
        errors = errors + 1
        #print('Too long or to short comment')

In [11]:
df_with_tensors = pd.DataFrame()
df_with_tensors['text'] = comment
df_with_tensors['tensor'] = array_of_mean_tensors
df_with_tensors['score'] = grade
df_with_tensors.to_csv('df_with_thensors_900.csv', sep = '|')

In [12]:
X = array_of_mean_tensors
y = df_with_tensors['score']

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [14]:
from sklearn.metrics import accuracy_score as ac
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error as mse
def metrics(y_test, y_pred, name_of_alg): 
    mae = np.mean(abs(y_test - y_pred))
    rmse = (mse(y_test, y_pred))**0.5
    print('\bMetrics and score for ', name_of_alg,':')
    print('Accuracy score from rmse:', 100 - np.mean(100 * (rmse / y_test)))
    print('Accuracy score from mae:', 100 - np.mean(100 * (mae / y_test)))
    print('classification_report: ')
    print(classification_report(y_test, y_pred))
    print('confusion matrix')
    print(confusion_matrix(y_test, y_pred))
    print('root mse')    
    print(rmse)
    print('root mae')    
    print(mae)

In [15]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)

In [16]:
y_pred = clf.predict(X_test)
print(metrics(y_test,y_pred, 'Random forest'))

Metrics and score for  Random forest :
Accuracy score from rmse: 66.85164127228063
Accuracy score from mae: 81.77815748100267
classification_report: 
              precision    recall  f1-score   support

           1       1.00      0.12      0.21        69
           2       0.62      0.76      0.68       177
           3       0.72      0.80      0.76       182

    accuracy                           0.68       428
   macro avg       0.78      0.56      0.55       428
weighted avg       0.72      0.68      0.64       428

confusion matrix
[[  8  47  14]
 [  0 135  42]
 [  0  36 146]]
root mse
0.6503054638104175
root mae
0.3574766355140187
None


In [19]:
for i in range(len(y_test)):
    index = y_test.index[i]
    print(df_with_tensors['text'][index])
    print('targeted_score: ', str(y[index]))
    print('predicted_score: ', str(y_pred[i]))
    print('-----------------------------')

я умерла и воскресла!... Вечная Книга! не только для чтения, но и для понимания - смыслов нереально много
targeted_score:  3
predicted_score:  3
-----------------------------
Почему никто никогда не говорил, что "Страна чудес без тормозов и Конец света" Мураками - это настоящий киберпанк?! Все признаки налицо: хакеры, мегакорпорации, всякие компьютерные и цифровые штуки с интеллектом, маргинальный образ жизни главного персонажа... Кайф! Только что добила эту книгу - я под впечатлением... Очень необычно, как и все у Мураками. До сих пор я была убеждена, что "Охота на овец" - это лучшее, что он написал. Открою свой маленький литературный секрет: книги Харуки читаю медленно и порционно - не больше одной в год. Иначе, если этого сумасшедшего японца читать залпом, в голове остается мешанина, а не странное звенящее послевкусие. Этот год под знаком "Страны чудес...". В общем, слов нет, одни эмоции.
targeted_score:  3
predicted_score:  2
-----------------------------
Здравствуйте, срочно куплю

Насколько я был в восторге от фильма, настолько же не понравилась книга. Почему-то все, что пытался прочесть у Кинга отдает плагиатом. Эта книга не исключение.Посмотрите лучше фильм.
targeted_score:  2
predicted_score:  2
-----------------------------
Мне в школе было непонятно, как можно прочитать целый толстый том "к завтрашнему уроку, а завтра пишем сочинение". Чтобы понять всю глубину этого произведения, потребовалось закончить школу. К сожалению, сейчас этой книги рядом нет, но как будет - обязательно перечитаю. Это классика, а к ней подход "галопом по Европам" абсолютно неприменим. Теперь, по прошествии лет, взгляд будет другим, но это и с любой книгой так - перечитывая через годы, смотришь с другого угла.
targeted_score:  3
predicted_score:  2
-----------------------------
В самом начале фильма нам заявляют, что более 30 000 человек из 21 миллиона пациентов, попадающих на операционный стол, так и не засыпают под наркозом. Так же как и зрители этого фильма, они вынуждены молча те

И пары лет не прошло с первой части, как группа товарищей под названием Фантастикфор Инк. уже снова зажигает на просторах голубой планеты, спасая мир в меру своих скромных сверхспособностей. То коня на скаку остановят, а то и горящую избу сделают ещё более горящей. На этом свете вообще мало чего меняется. Ботаник Рид по-прежнему пытается совершить обряд бракосочетания (дело дошло до того, что Сью изрядно призадумалась, что у суперпарочки могут народиться за супермутантики), братец новобрачной по-прежнему подогревает и без того перегретый рынок противоожоговой мази (и несгораемого бельишка для подружек), да и дядя Бен Гримм тоже не стоит на месте — его новая подруга жизни куда менее нервно реагирует на каменную волю близких, к тому же в упор не видит его недостатков, а потому всем хорошо и все бодры ака веселы. http://www.kinokadr.ru/articles/2007/06/17/ffour2.shtml
targeted_score:  2
predicted_score:  2
-----------------------------
Советую всем... Действительно хорошая книга
targeted_

Не последовательно... Не советую людям которых мутит от политики и взаимоотношений чиновников. Есть вроде-бы очень интересные места и идеи, сменяются нудностью и непоследовательностью. Мнение о книге неоднозначное. Вроде бы понимаешь что писал умный человек, но с другой стороны, не интересно, не моё.
targeted_score:  2
predicted_score:  2
-----------------------------
какой-то унылый вялотекущий онанизм. натужно и слабо. Эдакий иностранный Духлесс. тянется и тянется, зачем, для чего? для печатного объема на потребу издательства? осилила лишь 30% подобной галиматьи и концовку (а вдруг там был бы вдруг? не было!). И все беспросветно. Без эмоций. Философские опусы длинны и не к месту. Вычеркнешь эту натужную филосовию - остается только порнография. Уныло и бесцельно
targeted_score:  2
predicted_score:  2
-----------------------------
Боже какая тоска. Так и не дочитала, даже не захотела узнать чем все закончилось.
targeted_score:  1
predicted_score:  2
-----------------------------
НЕ зна

-----------------------------
Не знаю, я не в восторге. Думала, прочитаю что-то занимательное , ведущее к размышлениям. Оказалось, в книге перечислены истории из жизни... такие есть у всех... под заголовком "как я пил с друзьями"
targeted_score:  2
predicted_score:  2
-----------------------------
фильм просто отвратный. я, конечно, ничего особенного не ожидала от него, но то, что он будет так ужасен - даже предположить не могла!!! Отдаю должное рекламе - в нее было вложено немало... только за счет нее можно окупить фильм...
targeted_score:  1
predicted_score:  2
-----------------------------
Парадоксально, конечно, но очень странно смотрится фильм об "американском диком Западе", снятый американцами. Так и хочется сказать - эх, ребята, вы бы поучились у итальянцев снимать такие фильмы. Кстати, поучиться-то они попытались, но вышло как-то не очень. Ни рыба, ни мясо. Пафоса много, а толку мало. Неторопливость повествования и лаконичный сюжет а-ля Серджо Леоне в исполнении Сэма Рэйми боль

...

Не моё. Промучилась первую треть книги, не бросила только из-за того, что привыкла всегда дочитывать начатое. Потом втянулась, чтение пошло легче, но все равно читала я эту книгу очень долго. Не захватило. И еще постоянно раздражал главный герой: понимает, что его обманывают, с ним играют, но все равно упрямо, как баран, продолжает учавствовать в "спектакле", считая, что сможет все разгадать и переиграть "режиссера", вместо того, чтобы все бросив, сделать ему ("режиссеру") пакость.
targeted_score:  2
predicted_score:  2
-----------------------------
Моя первая книга в жанре фэнтези... Очень ее люблю, раз в год перечитываю и нахожу что-то новое. Недавно поняла, что считаю ВК реалистическим произведением. Дело, конечно, не в жизненных реалиях, но в психологической достоверности.
targeted_score:  3
predicted_score:  3
-----------------------------
Одна из самых лучших детских книжек. Она добрая, уютная (как домашние тапочки). Учит ценить жизнь и предостерегает.
targeted_score:  3
predicte

Лучшее произведение в жанре фентези, из прочитанных мной. Мощная и реалистичная магия, великолепный главный герой (именно такой, каким он должен быть в книге фентези), но больше всего меня поразила глубина чувств молодого мага к юной ларе. Несколькими короткими словами автор передаёт, то, что ,казалось бы, передать невозможно. Произведение выглядит неоконченным, хотябы потому, что Эрвин и Ди-и-ниль не испытали, по настоящему, свободы странствий по звёздным дорогам, а ещё потому, что, по законам жанра, они должы как минимум спасти мир :-) сражаясь под знаменем Союза Скальфа. Надеюсь это ещё впереди.
targeted_score:  3
predicted_score:  3
-----------------------------
В Дюне непостижимое макромышление автора наделяет и героев сверхчеловечностью - это победа ителлекта над врожденными магическими способностями и вообще американской идеей сверхчеловека, которому просто так случайно чето суперское досталось и он впоследствии желает употребить этот дар в пользу окружающим потомучто человек хо

На этой книге мой пыл угас и я дальше серию читать не собираюсь. Даже неудобно за патетический тон своего отзыва о первой книге, но из песни слов не выкинешь. Количество в качество не перешло. Того, что в первой книге показалось оригинальным и свежим, оказалось немного, в следующих двух оно разбавилось до прозрачности придурковатыми сюжетами и стало очевидным, что на красивом оформлении сцены и костюмах далеко не уедешь, нужна история, нужен сюжет, нужна психологическая достоверность. Да и оформление с костюмами (империя Ордусь) оказались прямолинейно перелицованным Советским Союзом. И если враждебность по отношению к Советскому Союзу мы можем объяснить реальными проблемами реальной страны, то враждебность врагов внутренних и внешних по отношению к выдуманной Ордуси авторы мотивируют дегенеративностью врагов, типа они просто больные, если не способны умилиться такой сообразной державой. Догматизм большевистского разлива. Раздражение вызывают романтические линии в книгах. Вторая жена-фр

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



2
-----------------------------
И слог как то не очень, и прописные истины автор как то корявенько подает. В общем, "велосипед" изобретенный Коэльо, вышел не очень. Не рекомендую как руководство к познанию мира.
targeted_score:  2
predicted_score:  2
-----------------------------
технологически идея книги завораживает введя читателя в ее суть автор жестоко расправляется с героями многочисленными, не греющими душу, которых не больно то и жалко книга хорошо спасает от приступов ТТБ (трамвайно-троллейбусного бешенства)
targeted_score:  2
predicted_score:  3
-----------------------------
Считаю,что зря потратила время.
targeted_score:  1
predicted_score:  3
-----------------------------
полная жесть. уничтожитель мозгов. издевательство и зомбирование людей. читайте лучше что-нибудь поумнее.
targeted_score:  1
predicted_score:  2
-----------------------------
Ни стиля, ни точки зрения я там не увидела. А вот потуги на Пелевинское поколение или Бегбедера - да. Но они смешны.
targeted_score: 